## Subword tokenization

 - основная идея: разделять текст не по словам, а по камим общим общим подсловам. Например `мирный` $\rightarrow$ [`мир`, `##н`, `##ый`]
 - можно ограничивать размер словаря
 - есть техники регуляризации
 - работает для агглютинативных языков 
 - можно обрабатывать незнакомые слова
 

## WordPiece

Процесс начинается со словаря, который состоит из букв. Затем на каждом шаге два элемента объеденяются такием образом, чтобы максимальным образом увеличить вероятность текста с точки зрения языковой модели модели.

## Byte-Pair Encoding (BPE)

Похоже на WordPiece, но считается количество повторений двух соседних элементов в словаре на каждом шаге. Элементы словаря с максимальной частотой сливаются.

In [ ]:
import tqdm.auto as tqdm
import gzip
from dataclasses import dataclass
from typing import Iterable

# Чтение файла данных
def read_texts(fn: str="data/news.txt.gz") -> Iterable[str]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield line.strip().split("\t")[2]

In [ ]:
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers import Tokenizer

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

tokenizer.pre_tokenizer = Whitespace()
tokenizer.normalizer = normalizer
tokenizer.decoder = WordPieceDecoder()

trainer = WordPieceTrainer(vocab_size=10000)

tokenizer.train_from_iterator(read_texts(), trainer=trainer)
tokenizer.save("data/news_tokenizer.json")

In [ ]:
tokenizer = Tokenizer.from_file("data/news_tokenizer.json")
res = tokenizer.encode("первому корове привет миру прививка")


In [ ]:
tokenizer.enable_padding(length=50)

import torch
res = tokenizer.encode_batch(["первому корове привет миру прививка", "хорошая погода"])
torch.tensor([x.ids for x in res])

In [ ]:
import torch.nn as nn

emb = nn.Embedding(10000, 100)
emb(torch.tensor([x.ids for x in tokenizer.encode_batch(["хорошая погода", "привет мир"])])).shape